### Piecemeal Clustering
A clustering approach without prior knowledge about data even number of clusters. It uses similarity and density of data to define number of clusters. 

### Reading the Paper
#### Introduction
- No noise: KMeans, SOM
- Density-based: DBSCAN
- Precedents: trial and errors, combinations of multiple methods
- What is lithofaces, 
- The algorithm utilizes the density-based clustering combining the concepts of hierachical clustering, model-based unsupervised learning and density-based data clustering. 

### Reference:
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9980364

In [11]:
import numpy as np
import pandas as pd
from sklearn import datasets
from scipy.cluster import hierarchy as sch

In [3]:
X, y = datasets.load_iris(as_frame=True, return_X_y=True)

In [4]:
data = X.to_numpy()
data.shape

(150, 4)

In [5]:
def merge_cluster(clusters, i, j):
    for t in range(len(clusters)):
        if clusters[t] == j:
            clusters[t] = clusters[i]
    return clusters

In [6]:
def ind_to_pair(d, index):
    b = 1 - (2 * d) 
    i = (-b - np.sqrt(b ** 2 - 8 * index)) // 2
    j = index + i * (b + i + 2) // 2 + 1
    return (int(i), int(j))  

In [7]:
def compute_dist_threshold(dmatrix, cutoff: float):
    dmin = min(dmatrix)
    dmax = max(dmatrix)
    return (1 - (dmin / dmax)) * cutoff

def compute_sim_threshold(smatrix, cutoff: float, diff_factor: float):
    dmin = min(smatrix)
    dmax = max(smatrix)
    return (1 - (dmin / dmax)) * cutoff * diff_factor

In [8]:
def compute_cluster_distances(dmatrix, smatrix, diff_factor: float):
    dmax = max(dmatrix)
    smax = max(smatrix)

    dmatrix = dmatrix / dmax
    dmatrix = dmatrix ** 2
    smatrix = smatrix / smax
    smatrix = diff_factor * (smatrix**2)

    return np.sqrt(dmatrix + smatrix)

In [33]:
def compute_cluster_centers(clusters, data):
    data = np.column_stack((data, clusters))
    df = pd.DataFrame(data, columns=["x1", "x2", "x3", "x4", "cluster"])
    df["cluster"] = df["cluster"].astype(int)
    df = df.groupby(by=["cluster"]).mean()
    return df.to_numpy()

In [34]:
def pre_clustering(clusters, cluster_centers, cutoff: float, diff_factor: float):
    dmatrix = sch.distance.pdist(cluster_centers, metric="euclidean")
    smatrix = sch.distance.pdist(cluster_centers, metric="cosine")
    dt = compute_dist_threshold(dmatrix, cutoff)
    st = compute_sim_threshold(smatrix, cutoff, diff_factor)
    tds = np.sqrt(dt**2 + st**2)
    cd = compute_cluster_distances(dmatrix, smatrix, diff_factor)
    dim = len(cluster_centers)

    print(f"Minimum distance: {tds}")
    sorted_cd = np.argsort(cd)
    
    for index in sorted_cd:
        d = cd[index]
        i, j = ind_to_pair(dim, index)
        if d <= tds:
            merge_cluster(clusters, i, j)

    return clusters

clusters = np.arange(0, len(data), 1)
clusters = pre_clustering(clusters, data, 0.015, 10)
centers = compute_cluster_centers(clusters, data)
print(centers[0])

Minimum distance: 0.15074813431681336
[5.006 3.428 1.462 0.246]


In [32]:
def train_som(data, W, alpha, N):
    """This is the modifed version of training Self-organizing map
    Args:
    - data: input data shape (m, n)
    - W: weight matrix, in piecemeal, the weight matrix is the cluster centers
    - alpha: learning rate
    - N: number of iterations
    """
    dmatrix = sch.distance.pdist(W, metric="euclidean")
    smatrix = sch.distance.pdist(W, metric="cosine")
    radius_dist = 
    radius_sim = 
    for i in range(N):


array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [6.36101695, 2.92542373, 5.05254237, 1.79830508],
       [5.53214286, 2.63571429, 3.96071429, 1.22857143],
       [6.3       , 3.3       , 6.        , 2.5       ],
       [7.475     , 3.125     , 6.3       , 2.05      ]])